In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.heatmap(df.isnull(), yticklabels=False, cbar=False)

In [ ]:
df.columns

In [ ]:
df['Loan_Status'].unique()

In [ ]:
df['Loan_Status'].nunique()

In [ ]:
df['Loan_Status']=df['Loan_Status'].replace('Y', 1)
df['Loan_Status']=df['Loan_Status'].replace('N', 0)

In [ ]:
df['Loan_Status'].unique()

In [ ]:
sns.countplot(df['Loan_Status'])

In [ ]:
df.isnull().sum()

In [ ]:
df['Dependents'].value_counts()

In [ ]:
df['Dependents'].unique()

In [ ]:
df['Dependents']=df['Dependents'].fillna('0')

In [ ]:
df['Dependents'].unique()

In [ ]:
df['Dependents']=df['Dependents'].replace('3+', 3)

In [ ]:
df['Dependents'].value_counts()

In [ ]:
df['Dependents'].value_counts()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10,5))
sns.countplot(df['Dependents'], hue=df['Loan_Status'])

In [ ]:
df.isnull().sum()

In [ ]:
df['Loan_Amount_Term'].describe()

In [ ]:
df['Loan_Amount_Term'].nunique()

In [ ]:
df['Loan_Amount_Term'].value_counts()

In [ ]:
df['Loan_Amount_Term'].mean()

In [ ]:
df['Loan_Amount_Term'].isnull().sum()

In [ ]:
plt.hist(df['Loan_Amount_Term'])

In [ ]:
df.isnull().sum()

In [ ]:
df['Credit_History'].unique()

In [ ]:
df['Credit_History'].value_counts()

In [ ]:
df['Credit_History']=df['Credit_History'].fillna(0.0)

In [ ]:
df['Credit_History'].unique()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10,5))
sns.countplot(df['Credit_History'], hue=df['Loan_Status'])

In [ ]:
df.isnull().sum()

In [ ]:
df['LoanAmount'].describe()

In [ ]:
df['LoanAmount']=df['LoanAmount'].fillna(df['LoanAmount'].mean())

In [ ]:
plt.hist(df['LoanAmount'])

In [ ]:
plt.boxplot(df['LoanAmount'])

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df['Self_Employed'].value_counts()

In [ ]:
df['Self_Employed'].unique()

In [ ]:
df['Self_Employed']=df['Self_Employed'].fillna(0)
df['Self_Employed']=df['Self_Employed'].replace('Yes', 1)

In [ ]:
df['Self_Employed']=df['Self_Employed'].replace('No', 0)

In [ ]:
df['Self_Employed'].unique()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10,5))
sns.countplot(df['Self_Employed'], hue=df['Loan_Status'])

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False, cbar=False)

In [ ]:
df.drop(['Loan_ID'], axis='columns', inplace=True)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df_categorical=df.select_dtypes(include=['object']).copy()
df_categorical.head(3)

In [ ]:
for i in df_categorical.columns:
    print(f'{i} : {df_categorical[i].unique()}')

In [ ]:
df_categorical['Dependents']=df_categorical['Dependents'].astype('int64')

In [ ]:
df_categorical['Dependents'][:5]

In [ ]:
df_categorical_copy=df_categorical.drop(['Dependents'], axis='columns')

In [ ]:
df_categorical_copy.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
scaler_label=LabelEncoder()
df_categorical_copy=df_categorical_copy.apply(LabelEncoder().fit_transform)

df_categorical_copy.head(5)

In [ ]:
df_new_categorical=pd.concat([df_categorical_copy,df_categorical['Dependents']], axis=1, join='inner')

In [ ]:
df_new_categorical.head()

In [ ]:
df_new_categorical.info()

In [ ]:
df_new_categorical['Dependents']=df_new_categorical['Dependents'].astype('int64')

In [ ]:
df_new_categorical.info()

In [ ]:
df_numeric=df.select_dtypes(exclude=['object']).copy()
df_numeric.head(3)

In [ ]:
df_numeric.info()

In [ ]:
for i in df_numeric.columns:
    sns.boxplot(df_numeric[i])
    plt.show()

In [ ]:
for i in df_numeric.columns:
    plt.hist(df_numeric[i])
    plt.xlabel(i)
    plt.show()

In [ ]:
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
df_numeric.shape

In [ ]:
print((df_numeric < (Q1 - 1.5 * IQR)) |(df_numeric > (Q3 + 1.5 * IQR)))

In [ ]:
df_numeric_remove_outlier = df_numeric[~((df_numeric < (Q1 - 1.5 * IQR)) |(df_numeric > (Q3 + 1.5 * IQR))).any(axis=1)]
df_numeric_remove_outlier.shape

In [ ]:
df_train=pd.concat([df_new_categorical, df_numeric_remove_outlier], axis=1, join='inner')

In [ ]:
df_train.head(5)

In [ ]:
df_train.info()

In [ ]:
x_data_train=df_train.drop(['Loan_Status'], axis='columns')
y_data_train=df_train.Loan_Status

In [ ]:
x_data_train.head(5)

In [ ]:
y_data_train[:5]

In [ ]:
x_data_train.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_train=StandardScaler()
scaler_train_data=scaler_train.fit_transform(x_data_train)

In [ ]:
scaler_train_data[:5]

In [ ]:
df_train_scaled=pd.DataFrame(data=scaler_train_data , columns=x_data_train.columns)

In [ ]:
df_train_scaled.head(5)

In [ ]:
y_data_train[:5]

# EDA for test data

In [ ]:
df_test = pd.read_csv("../input/loan-prediction-problem-dataset/test_Y3wMUE5_7gLdaTN.csv")

In [ ]:
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.heatmap(df_test.isnull(), yticklabels=False, cbar=False)

In [ ]:
df_test['Dependents'].value_counts()

In [ ]:
df_test['Dependents'].unique()

In [ ]:
df_test['Dependents']=df_test['Dependents'].replace('3+', '3')
df_test['Dependents']=df_test['Dependents'].fillna('0')

In [ ]:
df_test['Dependents'].unique()

In [ ]:
df_test['Dependents'].value_counts()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10,5))
sns.countplot(df_test['Dependents'])

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test['Loan_Amount_Term'].describe()

In [ ]:
df_test['Loan_Amount_Term'].nunique()

In [ ]:
df_test['Loan_Amount_Term'].nunique()

In [ ]:
df_test['Loan_Amount_Term'].value_counts()

In [ ]:
df_test['Loan_Amount_Term'].mean()

In [ ]:
df_test['Loan_Amount_Term']=df_test['Loan_Amount_Term'].fillna(df_test['Loan_Amount_Term'].mean())

In [ ]:
df_test['Loan_Amount_Term'].isnull().sum()

In [ ]:
plt.hist(df_test['Loan_Amount_Term'])

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test['Credit_History'].unique()

In [ ]:
df_test['Credit_History'].value_counts()

In [ ]:
df_test['Credit_History']=df_test['Credit_History'].fillna(0.0)

In [ ]:
df_test['Credit_History'].unique()

In [ ]:
df_test['Credit_History'].isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test['LoanAmount'].describe()

In [ ]:
df_test['LoanAmount']=df_test['LoanAmount'].fillna(df_test['LoanAmount'].mean())

In [ ]:
df_test['LoanAmount'].isnull().sum()

In [ ]:
plt.hist(df_test['LoanAmount'])

In [ ]:
plt.boxplot(df_test['LoanAmount'])

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test['Self_Employed'].value_counts()

In [ ]:
df_test['Self_Employed'].unique()

In [ ]:
df_test['Self_Employed']=df_test['Self_Employed'].fillna(0)
df_test['Self_Employed']=df_test['Self_Employed'].replace('Yes', 1)
df_test['Self_Employed']=df_test['Self_Employed'].replace('No', 0)

In [ ]:
df_test['Self_Employed'].unique()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(5,5))
sns.countplot(df_test['Self_Employed'])

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.dropna(inplace=True)

In [ ]:
sns.heatmap(df_test.isnull(),yticklabels=False, cbar=False)

In [ ]:
df_test.drop(['Loan_ID'], axis='columns', inplace=True)

In [ ]:
df_test.columns

In [ ]:
df_test.info()

In [ ]:
df_categorical_test=df_test.select_dtypes(include=['object']).copy()
df_categorical_test.head(3)

In [ ]:
for i in df_categorical_test.columns:
    print(f'{i} : {df_categorical_test[i].unique()}')

In [ ]:
from sklearn.preprocessing import LabelEncoder
scaler_label=LabelEncoder()
df_categorical_test=df_categorical_test.apply(LabelEncoder().fit_transform)

df_categorical_test.head(5)

In [ ]:
df_categorical_test['Dependents'].unique()

In [ ]:
df_categorical_test.info()

In [ ]:
df_numeric_test=df_test.select_dtypes(exclude=['object']).copy()
df_numeric_test.head(3)

In [ ]:
df_final_test=pd.concat([df_categorical_test, df_numeric_test], axis=1, join='inner')

In [ ]:
df_final_test.info()

In [ ]:
df_final_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_test=StandardScaler()
scaler_test_data=scaler_test.fit_transform(df_final_test)

In [ ]:
scaler_test_data[:5]

In [ ]:
df_test_scaled=pd.DataFrame(data=scaler_test_data , columns=df_final_test.columns)

In [ ]:
df_test_scaled.head(5)

In [ ]:
#train_test_split.

from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test= train_test_split(df_train_scaled,y_data_train, test_size=0.2)

In [ ]:
X_train.shape , X_test.shape , y_train.shape , y_test.shape

In [ ]:
#model creation by using DecisionTreeClassifier

from sklearn.tree import DecisionTreeClassifier
model_DT=DecisionTreeClassifier(criterion='entropy')
model_DT.fit(X_train,y_train)

In [ ]:
y_pred_DT=model_DT.predict(X_test)

In [ ]:
y_pred_DT[:5]

In [ ]:
y_test[:5]

In [ ]:
model_DT.score(X_test,y_test)

# using gridsearchcv and crossvalidation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import  DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix





In [ ]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20,25,30,40],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10,15,20,25,30]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear'),
        'params': {
            'C': [1,5,10,15,20,25]
        }
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params':{
            'criterion':['gini','entropy']
        }
    }
}

In [ ]:
#cv=5
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df_score = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_score

### Hence we conclude that svm plays the better role with c=1 and kernel =rbf

In [ ]:
model=SVC(kernel='rbf')
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred[:5]

In [ ]:
y_train[:5]

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
y_pred_test_data=model.predict(df_test_scaled)

In [ ]:
y_pred_test_data[:5]